# Desafio: Consumo de Dados para Previsão do Tempo dos Municípios do ES.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo dos municípios do ES. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/estados/32/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada município. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do ES, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do ES. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Vila Velha. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e zipar juntamente com os CSVs das duas tabelas e postar no grupo do Teams.



-----------------------------------------------
### Caso queira utilizar o colab do google para executar o script Spark descomente as linhas abaixo

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2023-01-24 12:13:05--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  9.39MB/s    in 1.4s    

2023-01-24 12:13:07 (9.39 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[],"uri":"/api/tunnels"}


---------------------------------------------------

In [4]:
# use o config [.config('spark.ui.port', '4050') \] caso queira ver o spark UI no colab, caso contrário pode retirar esta configuração
!pip install -q findspark
!pip install -q unidecode
import findspark
findspark.init()

import requests
import json
import unidecode
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .config('spark.ui.port', '4050') \
      .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 11.2 MB/s eta 0:00:00


In [5]:
# Buscar cidades do Espírito Santo
# TODO
request = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados/32/municipios")
#print(request.content)
dadosApi = json.loads(request.content)
print(dadosApi)


# Criar data frame com as cidades
# TODO
js = request.json()
df = spark.createDataFrame(js)


# Criar view com as cidades
# TODO
df.createOrReplaceTempView("cities")
cidades = spark.sql('select nome from cities where nome not like "Água Doce do Norte"')
cidades.show(5)

[{'id': 3200102, 'nome': 'Afonso Cláudio', 'microrregiao': {'id': 32007, 'nome': 'Afonso Cláudio', 'mesorregiao': {'id': 3203, 'nome': 'Central Espírito-santense', 'UF': {'id': 32, 'sigla': 'ES', 'nome': 'Espírito Santo', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}, 'regiao-imediata': {'id': 320002, 'nome': 'Afonso Cláudio - Venda Nova do Imigrante - Santa Maria de Jetibá', 'regiao-intermediaria': {'id': 3201, 'nome': 'Vitória', 'UF': {'id': 32, 'sigla': 'ES', 'nome': 'Espírito Santo', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}}, {'id': 3200136, 'nome': 'Águia Branca', 'microrregiao': {'id': 32002, 'nome': 'Nova Venécia', 'mesorregiao': {'id': 3201, 'nome': 'Noroeste Espírito-santense', 'UF': {'id': 32, 'sigla': 'ES', 'nome': 'Espírito Santo', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}, 'regiao-imediata': {'id': 320005, 'nome': 'Colatina', 'regiao-intermediaria': {'id': 3203, 'nome': 'Colatina', 'UF': {'id': 32, 'sigla': 'ES', 'nome': 'Espírito 

In [6]:
# Buscar previsão do tempo para as cidades
# TODO
API_KEY = "fbd35c220a56788cc751f0fc9b538af1"
#city_name = "Afonso Cláudio"



#requisicao_prev = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={'Vila Velha'}&cnt={'1'}&appid={API_KEY}")

#requisicao_dic2 = json.loads(requisicao_prev.content)
#print(requisicao_dic2)

#descricao = requisicao_dic2['list'][1]['weather'][0]['description']
#temperatura = requisicao_dic2['list'][1]['main']['temp'] -273.15
#codigo_cidade = requisicao_dic2['cod']
#print(descricao, codigo_cidade, temperatura)

###

###Removi algumas cidades pois a API não encontrou
cidades5 = spark.sql('''select nome from cities where nome not like "Água Doce do Norte" limit 30
                       ''')
cidade_collect = cidades5.collect()
print(cidade_collect)


for row in cidade_collect:
  city_name = (row['nome'])
  URL_PREVISAO = f"https://api.openweathermap.org/data/2.5/forecast?q={city_name}&lang=pt_br&appid={API_KEY}"
  requisicao_prev = requests.get(URL_PREVISAO)
  requisicao_dic = json.loads(requisicao_prev.content)
  
  print(requisicao_dic)
  
  codigo_ret = requisicao_dic['cod']
  condicao_tempo = requisicao_dic['list'][0]['weather'][0]['description']
  temperaturaK = requisicao_dic['list'][0]['main']['temp']
  temperaturaC = requisicao_dic['list'][0]['main']['temp'] -273.15
  temp_min = requisicao_dic['list'][0]['main']['temp_min'] -273.15
  temp_max = requisicao_dic['list'][0]['main']['temp_max'] -273.15
  sensacao = requisicao_dic['list'][0]['main']['feels_like'] -273.15
  umidade = requisicao_dic['list'][0]['main']['humidity']
  data = requisicao_dic['list'][0]['dt_txt']
  print("Cidade: "+city_name + "  Condição do Tempo: "+condicao_tempo+ " Temperatura C: "+str(temperaturaC) + " umidade: " +str(umidade) + " Data: " +data  )






[Row(nome='Afonso Cláudio'), Row(nome='Águia Branca'), Row(nome='Alegre'), Row(nome='Alfredo Chaves'), Row(nome='Alto Rio Novo'), Row(nome='Anchieta'), Row(nome='Apiacá'), Row(nome='Aracruz'), Row(nome='Atílio Vivácqua'), Row(nome='Baixo Guandu'), Row(nome='Barra de São Francisco'), Row(nome='Boa Esperança'), Row(nome='Bom Jesus do Norte'), Row(nome='Brejetuba'), Row(nome='Cachoeiro de Itapemirim'), Row(nome='Cariacica'), Row(nome='Castelo'), Row(nome='Colatina'), Row(nome='Conceição da Barra'), Row(nome='Conceição do Castelo'), Row(nome='Divino de São Lourenço'), Row(nome='Domingos Martins'), Row(nome='Dores do Rio Preto'), Row(nome='Ecoporanga'), Row(nome='Fundão'), Row(nome='Governador Lindenberg'), Row(nome='Guaçuí'), Row(nome='Guarapari'), Row(nome='Ibatiba'), Row(nome='Ibiraçu')]
{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1674572400, 'main': {'temp': 298.69, 'feels_like': 299.6, 'temp_min': 298.69, 'temp_max': 300.05, 'pressure': 1012, 'sea_level': 1012, 'grnd_level'

In [7]:
# Criar data frame com as previsões
# TODO
dtprev = pd.DataFrame(columns = ['Cidade', 'Data', 'Temperatura', 'Temperatura_Min', 'Temperatura_Max', 'Sensação Térmica', 'Umidade', 'Condição do Tempo'    ])
dtprev

for row in cidade_collect:
  city_name = (row['nome'])
  URL_PREVISAO = f"https://api.openweathermap.org/data/2.5/forecast?q={city_name}&lang=pt_br&appid={API_KEY}"
  requisicao_prev = requests.get(URL_PREVISAO)
  requisicao_dic = json.loads(requisicao_prev.content)

  
  codigo_ret = requisicao_dic['cod']
  condicao_tempo = requisicao_dic['list'][0]['weather'][0]['description']
  temperaturaK = requisicao_dic['list'][0]['main']['temp']
  temperaturaC = requisicao_dic['list'][0]['main']['temp'] -273.15
  temp_min = requisicao_dic['list'][0]['main']['temp_min'] -273.15
  temp_max = requisicao_dic['list'][0]['main']['temp_max'] -273.15
  sensacao = requisicao_dic['list'][0]['main']['feels_like'] -273.15
  umidade = requisicao_dic['list'][0]['main']['humidity']
  data = requisicao_dic['list'][0]['dt_txt']
  dtprev = dtprev.append({'Cidade':city_name, 'Data': data, 'Temperatura':temperaturaC, 'Temperatura_Min':temp_min, 'Temperatura_Max':temp_max,
                 'Sensação Térmica':sensacao, 'Umidade': umidade, 'Condição do Tempo': condicao_tempo}, ignore_index = True)


dtprev


,Cidade,Data,Temperatura,Temperatura_Min,Temperatura_Max,Sensação Térmica,Umidade,Condição do Tempo
0,Afonso Cláudio,2023-01-24 15:00:00,25.54,25.54,26.90,26.45,88,chuva leve
1,Águia Branca,2023-01-24 15:00:00,27.89,27.89,31.00,31.54,78,chuva leve
2,Alegre,2023-01-24 15:00:00,25.49,25.49,29.14,26.34,86,chuva leve
3,Alfredo Chaves,2023-01-24 15:00:00,26.39,26.39,29.20,26.39,87,chuva leve
4,Alto Rio Novo,2023-01-24 15:00:00,25.71,25.71,27.50,26.35,77,chuva leve
5,Anchieta,2023-01-24 15:00:00,27.29,27.29,30.92,27.56,48,céu limpo
6,Apiacá,2023-01-24 15:00:00,26.56,26.56,29.03,26.56,83,nublado
7,Aracruz,2023-01-24 15:00:00,25.27,25.27,30.69,26.05,84,nublado
8,Atílio Vivácqua,2023-01-24 15:00:00,24.73,24.73,27.93,25.56,88,chuva leve
9,Baixo Guandu,2023-01-24 15:00:00,27.63,27.63,29.21,31.33,81,chuva moderada


In [8]:
# Criar view com as previsões
# TODO
dtprev = spark.createDataFrame(dtprev)
dtprev.createOrReplaceTempView("previsoes")
previsoes = spark.sql('select * from previsoes')
previsoes.show()

+--------------------+-------------------+------------------+------------------+------------------+------------------+-------+-----------------+
|              Cidade|               Data|       Temperatura|   Temperatura_Min|   Temperatura_Max|  Sensação Térmica|Umidade|Condição do Tempo|
+--------------------+-------------------+------------------+------------------+------------------+------------------+-------+-----------------+
|      Afonso Cláudio|2023-01-24 15:00:00| 25.54000000000002| 25.54000000000002|26.900000000000034|26.450000000000045|     88|       chuva leve|
|        Águia Branca|2023-01-24 15:00:00|27.890000000000043|27.890000000000043|              31.0| 31.54000000000002|     78|       chuva leve|
|              Alegre|2023-01-24 15:00:00| 25.49000000000001| 25.49000000000001|29.140000000000043|26.340000000000032|     86|       chuva leve|
|      Alfredo Chaves|2023-01-24 15:00:00|26.390000000000043|26.390000000000043|29.200000000000045|26.390000000000043|     87|    

In [ ]:
# Criar DF da Tabela 2
# TODO

In [9]:
# Exportar CSVs
# TODO
df = dtprev.toPandas()
df.to_csv("Previsoes.csv")